<a href="https://colab.research.google.com/github/jkranyak/project_3/blob/main/project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tensorflow


Step 1: Data Preparation and Preprocessing
DICOM Images

DICOM Handling: Use pydicom to read DICOM files, which contain both the images and embedded metadata. For each DICOM file, extract the image data for preprocessing and the metadata for analysis.
Image Preprocessing: Since DICOM images might not be uniformly sized, you may need to resize them to a standard size (e.g., 1024x1024) to match the TFRecord images, normalize pixel values, and potentially augment the data to improve model robustness.
JPEG and TFRecord Images

JPEG Handling: Directly read and preprocess (resize if necessary, normalize, augment).
TFRecord Handling: Extract and preprocess. TFRecord files are already uniformly sized, which simplifies preprocessing.
Metadata

CSV Metadata: Load the metadata from CSV files. Clean and preprocess this data, which involves handling missing values, normalizing or scaling numerical features, and encoding categorical features.



In [4]:
!pip install isic-cli


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.9 MB/s eta 0:00:00


In [5]:
! isic user login

visit the following url in a browser:
https://api.isic-archive.com/oauth/authorize?response_type=code&client_id=RpCzc4hFjv5gOJdM2DM2nBdokOviOh5ne63Tpn7Q&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=szpDCj95PH5xL0sjBRruQu8crfkXyc&code_challenge=EEfrSJ2Pzm4yD2XT5ADFX7tEjX2nOTMryjCU47meSr0&code_challenge_method=S256
enter the code shown in your browser: Ww7huyqCiRXfHvnJakpInteV4eIi8a
Success!


Step 2: Query the Dataset
Endpoints: Use the API endpoints to query the dataset. Common operations include listing available images, retrieving image metadata, and downloading images.
Filtering: Utilize query parameters to filter the dataset based on your criteria, such as diagnosis, image type, or other metadata.

In [6]:
!pip install isic-cli


In [28]:
! pip install kaggle

In [31]:
!kaggle competitions download -c titanic

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [7]:
!isic collection list


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ ID  ┃ Name                                          ┃ Public ┃ Pinned ┃ Locked ┃ DOI             ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 249 │ BCN20000                                      │ True   │ False  │ False  │ None            │
│ 61  │ Challenge 2016: Test                          │ True   │ True   │ True   │ None            │
│ 74  │ Challenge 2016: Training                      │ True   │ True   │ True   │ None            │
│ 69  │ Challenge 2017: Test                          │ True   │ True   │ True   │ None            │
│ 60  │ Challenge 2017: Training                      │ True   │ True   │ True   │ None            │
│ 71  │ Challenge 2017: Validation                    │ True   │ True   │ True   │ None            │
│ 64  │ Challenge 2018: Task 1-2: Test                │ True   │ True   │ True   │ None    

Step 3: Download Images and Metadata
Batch Processing: Write a script to automatically download batches of images and their associated metadata.

Data Storage will utilize wounting a drive in google and creating files for the train, test, and validation images.

In [48]:
import requests
import os
import pandas as pd
import tensorflow as tf
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import img_to_array, load_img

from google.colab import drive
from PIL import Image

In [9]:
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [32]:
# code block from kaggle documentation & chatgpt
# Specify the current path of kaggle.json file
current_path = '/content/kaggle.json'

# Desired path where the Kaggle API expects the kaggle.json file
desired_path = '/root/.kaggle/kaggle.json'

os.makedirs(os.path.dirname(desired_path), exist_ok=True)
os.rename(current_path, desired_path)

# Set the file's permissions to avoid a permissions error
os.chmod(desired_path, 600)


find the data set on kaggle


In [35]:
!kaggle datasets download -d andrewmvd/isic-2019


100% 9.10G/9.10G [05:18<00:00, 29.9MB/s]
100% 9.10G/9.10G [05:18<00:00, 30.7MB/s]


In [36]:
!unzip -q isic-2019.zip


Load the metadata

and

Explore the Metadata CSV: Load the metadata.csv files for training, test, and validation sets to understand the structure and types of data available. This step is crucial for preprocessing and feature selection.

In [40]:
# Load the metadata
metadata = pd.read_csv('/content/ISIC_2019_Training_Metadata.csv')

# Display the first few rows of each DataFrame
metadata.head()


,image,age_approx,anatom_site_general,lesion_id,sex
0,ISIC_0000000,55.0,anterior torso,NaN,female
1,ISIC_0000001,30.0,anterior torso,NaN,female
2,ISIC_0000002,60.0,upper extremity,NaN,female
3,ISIC_0000003,30.0,upper extremity,NaN,male
4,ISIC_0000004,80.0,posterior torso,NaN,male


In [43]:
ground_truth = pd.read_csv('/content/ISIC_2019_Training_GroundTruth.csv')
ground_truth

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# Load the CSV files
ground_truth = pd.read_csv('/content/ISIC_2019_Training_GroundTruth.csv')
metadata = pd.read_csv('/content/ISIC_2019_Training_Metadata.csv')

image_dir = '/content/ISIC_2019_Training_Images'
ground_truth['image_path'] = ground_truth['image'].apply(lambda x: os.path.join(image_dir, x + '.jpg'))

# Merge the ground_truth with metadata if necessary
full_metadata = pd.merge(ground_truth, metadata, on='image', how='left')  # Adjust 'on' parameter as needed
full_metadata

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,image_path,age_approx,anatom_site_general,lesion_id,sex
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,55.0,anterior torso,NaN,female
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,30.0,anterior torso,NaN,female
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,60.0,upper extremity,NaN,female
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,30.0,upper extremity,NaN,male
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_000000...,80.0,posterior torso,NaN,male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007324...,85.0,head/neck,BCN_0003925,female
25327,ISIC_0073248,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007324...,65.0,anterior torso,BCN_0001819,male
25328,ISIC_0073249,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007324...,70.0,lower extremity,BCN_0001085,male
25329,ISIC_0073251,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/content/ISIC_2019_Training_Images/ISIC_007325...,55.0,palms/soles,BCN_0002083,female


In [45]:
full_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25331 entries, 0 to 25330
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   image                25331 non-null  object 
 1   MEL                  25331 non-null  float64
 2   NV                   25331 non-null  float64
 3   BCC                  25331 non-null  float64
 4   AK                   25331 non-null  float64
 5   BKL                  25331 non-null  float64
 6   DF                   25331 non-null  float64
 7   VASC                 25331 non-null  float64
 8   SCC                  25331 non-null  float64
 9   UNK                  25331 non-null  float64
 10  image_path           25331 non-null  object 
 11  age_approx           24894 non-null  float64
 12  anatom_site_general  22700 non-null  object 
 13  lesion_id            23247 non-null  object 
 14  sex                  24947 non-null  object 
dtypes: float64(10), object(5)
memory usa

In [46]:
# Fill missing values for 'age_approx' with its median
full_metadata['age_approx'].fillna(full_metadata['age_approx'].median(), inplace=True)

# For categorical data, fill missing values with 'unknown'
full_metadata['anatom_site_general'].fillna('unknown', inplace=True)
full_metadata['sex'].fillna('unknown', inplace=True)


In [47]:
full_metadata = pd.get_dummies(full_metadata, columns=['anatom_site_general', 'sex'])


combining the images and the cleaned data


In [50]:
# Image input branch
image_input = Input(shape=(224, 224, 3))
x = Conv2D(32, (3, 3), activation='relu')(image_input)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
image_branch = Dense(128, activation='relu')(x)

# Metadata input branch
metadata_input = Input(shape=(15,))
y = Dense(32, activation='relu')(metadata_input)
metadata_branch = Dense(64, activation='relu')(y)

# Combine branches
combined = concatenate([image_branch, metadata_branch])
z = Dense(256, activation='relu')(combined)
# 9 diagnostic categories
output = Dense(9, activation='softmax')(z)

# define the model
model = Model(inputs=[image_input, metadata_input], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


Evaluate the model

In [51]:
# Evaluate the model on the test set
test_generator = create_custom_data_generator(X_test, y_test, batch_size=32)
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(X_test) // 32)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


NameError: name 'create_custom_data_generator' is not defined

Step 2: Merging Image and Metadata Features
To incorporate both image data and metadata in your model, consider a dual-input neural network architecture. One branch of the network will process the image data (using CNN layers), and the other branch will process the structured metadata (using dense layers). The outputs of these two branches can be concatenated and fed into additional layers to perform the final prediction.



Step 3: Model Architecture
Image Branch: Utilize a pre-trained CNN (like ResNet or Inception) as the base for feature extraction from images. Fine-tune this model with your specific dataset.
Metadata Branch: Design a simple feedforward neural network that takes in the processed metadata as input and outputs a representation of this data.
Combination and Prediction: Merge the outputs of both branches and feed them through one or more dense layers to produce a final prediction.

Step 4: Training
Loss Function: Use binary cross-entropy as the loss function since this is a binary classification task.
Metrics: Track accuracy and AUC (Area Under the Curve) to evaluate model performance.
Validation: Use a separate validation set (not seen by the model during training) to fine-tune hyperparameters and avoid overfitting.

Step 5: User Interface (UI)
A simple web-based UI (using Flask or Streamlit) allows users to upload lesion images, input relevant metadata, and receive a prediction. The UI should:

Enable image upload and metadata entry.
Preprocess the input data similarly to the training data.
Display the model's malignancy probability.

4. Integrate APIs

If using OpenAI's GPT-3, you'll interact with it through its API, sending requests with the classification results and receiving generated text.
If your application needs to store or retrieve data, consider integrating a database and accessing it through a REST API.


5. Build the Application Logic

Develop a user interface where users can upload photos of skin growths.
After uploading, the photo is sent to the image classification model to determine the type of growth.
The classification result is used to generate a prompt for the ChatGPT-like model, which then creates an informative response.
Display the response to the user along with any additional advice or recommendations.